In [1]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv("GrandEst",sep=";")

In [38]:
df.head(10)

,REGION,NUMMR,ACHLR,AEMM,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,...,TP,TRANS,TYPC,TYPFC,TYPL,TYPMD,TYPMR,UR,VOIT,WC
0,44,1,3,2008,9,58,64,57,55,1959,...,Z,Z,3.0,2,2.0,500.0,44.0,1.0,1.0,Z
1,44,1,3,2008,9,69,79,68,65,1948,...,Z,Z,3.0,2,2.0,500.0,44.0,1.0,1.0,Z
2,44,2,4,1977,6,70,79,69,65,1948,...,Z,Z,1.0,Z,1.0,22.0,12.0,0.0,1.0,Z
3,44,3,3,2012,9,30,29,29,25,1985,...,1,4,3.0,2,2.0,300.0,41.0,0.0,1.0,Z
4,44,3,3,2012,9,30,29,29,25,1985,...,1,4,3.0,2,2.0,300.0,41.0,0.0,1.0,Z
5,44,4,6,2014,9,30,29,29,25,1988,...,1,4,3.0,Z,2.0,11.0,11.0,1.0,1.0,Z
6,44,5,7,2016,9,1,2,0,0,2016,...,Z,Z,2.0,2,1.0,301.0,41.0,1.0,2.0,Z
7,44,5,7,2016,9,26,29,25,25,1991,...,1,4,2.0,2,1.0,301.0,41.0,1.0,2.0,Z
8,44,5,7,2016,9,37,39,36,35,1980,...,1,4,2.0,2,1.0,301.0,41.0,1.0,2.0,Z
9,44,6,6,2012,9,42,54,41,40,1974,...,1,4,3.0,2,2.0,300.0,41.0,1.0,2.0,Z


# An imortant note : 

For __INPER__ attribute the values found in the dataset differes from the ones in the signification pdf!!!!
Where in the pdf they are Y and Z while in the data set they are numbers!
So i considered that the numbers are indication to the number of person in the household
<BR>
For example 3 indicated the there are 3 people who live in the household

In [46]:
df.INPER.unique()

array([ 2.,  1.,  3.,  4.,  6.,  5.,  8.,  7.,  9., 10., 11., 15., 14.,
       19., 13., 12., 38., 16., 17., nan])

In [40]:
# for i in df.columns:
#     print(i,df[i].unique())

In [15]:
# cols=["AGER20", "ANARR", "ASCEN", "BAIN",
# "BATI", "CATL", "CHOS", "CLIM", "CMBL", "COUPLE", "CS1", "CUIS", "DEROU", "DIPL_15", "EAU", "EGOUL",
# "ELEC", "EMPL", "ETUD", "GARL", "ILTUU", "IMMI", "INAT", "INFAM", "INPER", "MOCO", "MODV", "NA38",
# "NATC", "NATNC", "NBPI", "NPERR", "RECH", "SANI", "SEXE", "SFM", "STOCD", "TACT", "VOIT", "WC"]

## Dictionaries 

In [49]:
# A 2 dimensional dictionary is created which have as a key, 
# the attribute name and as a value, another dictionary of 
# which each key represents a unique value of its key attribute
# with a corresponding unique integer as a value in the second dictionary.
#at the same time a reverse dictionary is created for decoding use

def generate_dic(df,columns):
    ind=0
    dic_enc={}
    dic_dec={}
    for column in columns:
        for val in df[column].unique():
            if(dic_enc.get(column) is None):
                dic_enc[column]={}
            dic_enc[column][str(val)]=ind
            dic_dec[str(ind)]="('"+column+"', '"+str(val)+"')"
            ind+=1
    return (dic_enc,dic_dec)


## Encoder for the DATA SET FILE

In [2]:

# Then for each transaction in the dataset a 
# mapping was created from the unique items in 
# the dataset to integers so that each item corresponds 
# to a unique integer according to the dictionary created.

def encode_line(line,dic):
    enc=""
    for ind,val in line.items():
        if enc!="":
            enc+=" "
        enc+=str(dic[ind][str(val)])
    return enc


def encoder(input_file,columns,output_file,dic_enc):
    df=pd.read_csv(input_file,sep=";")
    f=open(output_file,"w")
    for _,val in df[columns].iterrows():
        f.write(encode_line(val,dic_enc))
        f.write("\n")
    f.close()
    


## Decoder for ITEMSETS

In [ ]:
######################for itemsets#################################

def decode_line(line,dic_dec):
    decoded_line=""
    line_array=line.strip().split("#")
    items=line_array[0].strip().split(" ")
    for item in items:
        
        decoded_line+=dic_dec[item]+" "
    decoded_line+=str("#"+line_array[1])
    return decoded_line

def decoder(inputfile,dic,outputfile):
    file=open(inputfile, 'r')
    lines=file.readlines()
   
    with open(outputfile, 'w') as f:
        for line in lines:
            decoded_line=decode_line(line,dic)
            print(decoded_line, file=f)

## Decoder for ASSOCIATION RULES

In [3]:
# 35 ==> 13 #SUP: 907668 #CONF: 0.8936299668507417
def decode_line_association_rules(line,dic):
    decoded_line=""
    line_array=line.strip().split("#")
    items=line_array[0].strip().split("==>")
    for item in items[0].strip().split(" "):
        decoded_line+=dic_dec[item]+" "
    decoded_line+="==> "
    for item in items[1].strip().split(" "):
        decoded_line+=dic_dec[item]+" "
    decoded_line+=str("#"+line_array[1])
    decoded_line+=str(" #"+line_array[2])

    return decoded_line


def decoder_association_rules(inputfile,dic,outputfile):
    file=open(inputfile, 'r')
    lines=file.readlines()
   
    with open(outputfile, 'w') as f:
        for line in lines:
            decoded_line=decode_line_association_rules(line,dic)
            print(decoded_line, file=f)

## Selecting the attributes and generating the dictionaries

In [58]:
cols=["AGER20", "ANARR","DEROU" ,"DIPL_15", "EMPL", "ETUD", "GARL", "ILTUU", "IMMI", "INAT", "INFAM", "INPER", "MODV",
"NATC", "NBPI", "NPERR", "RECH", "SFM", "TACT", "VOIT","TYPMR","TRANS","PNAI12", "CS2","TYPL"]

In [59]:
len(cols)

25

In [32]:
dic_enc,dic_dec=generate_dic(df,cols)

## Encoding the dataset

In [33]:
encoder("GrandEst",cols,"results/encoded_dataset.txt",dic_enc)

## Decoding itemsets for different supports

In [15]:
decoder('results/encoded_itemsets_minsup30.txt',dic_dec,'results/decoded_itemsets_minsup30.txt')

In [38]:
decoder('results/encoded_itemsets_minsup40.txt',dic_dec,'results/decoded_itemsets_minsup40.txt')

In [39]:
decoder('results/encoded_itemsets_minsup60.txt',dic_dec,'results/decoded_itemsets_minsup60.txt')

In [52]:
decoder('results/encoded_itemsets_minsup50.txt',dic_dec,'results/decoded_itemsets_minsup50.txt')

## Decoding the resulted association rules

In [43]:
decoder_association_rules('results/encoded_association_rules_minsup30_confi_80.txt',dic_dec,'results/decoded_association_rules_minsup30_confi_80.txt')

In [54]:
decoder_association_rules('results/encoded_association_rules_minsup60_confi_80.txt',dic_dec,'results/decoded_association_rules_minsup60_confi_80.txt')

In [60]:
decoder_association_rules('results/encoded_association_rules_minsup40_confi_80.txt',dic_dec,'results/decoded_association_rules_minsup40_confi_80.txt')

In [62]:
decoder_association_rules('results/encoded_association_rules_minsup50_confi_80.txt',dic_dec,'results/decoded_association_rules_minsup50_confi_80.txt')